In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 50)

In [2]:
TRAIN_DATA_PATH = './data/train_data.csv'
TRAIN_TARGET_PATH = './data/train_target.csv'
TEST_DATA_PATH = './data/test_data.csv'
TEST_TARGET_PATH = './data/test_target.csv'

In [3]:
data = pd.read_csv(TRAIN_DATA_PATH)
target = pd.read_csv(TRAIN_TARGET_PATH)

Соединяю data и target для дальнейщей обработки

In [4]:
df = pd.concat([data, target], axis=1)
df.head()

,n_rooms,territorial_division,district,total_area,living_space,kitchen_area,floor,total_floors,year_of_built,celling_height,renovation_type,outside_view,n_balconies,n_loggias,n_sep_bathrooms,n_comb_bathrooms,building_type,overlap_type,n_pass_lifts,n_serv_lifts,heating_type,home_emergency,parking_type,garbage_chute,mins_to_subway_by_walk,mins_to_subway_by_car,mins_to_subway_by_trans,price
0,3.0,ЦАО,Пресненский,132.0,NaN,NaN,2,13,2017.0,NaN,Дизайнерский,Во двор,0,0,0,2,Монолитный,Железобетонные,0,0,Центральное,Нет,Подземная,NaN,10.0,NaN,NaN,130000000.0
1,2.0,ЗАО,Тропарево-Никулино,58.4,37.0,10.0,15,16,1972.0,NaN,Евроремонт,Во двор,0,0,0,1,Панельный,Железобетонные,0,0,Центральное,Нет,NaN,Есть,10.0,NaN,NaN,18700000.0
2,1.0,СЗАО,Митино,27.5,17.0,5.0,3,9,1999.0,2.7,Косметический,Во двор,0,0,0,1,Панельный,Железобетонные,0,0,Центральное,Нет,NaN,NaN,20.0,NaN,NaN,7699000.0
3,2.0,СВАО,Лианозово,56.2,31.6,10.0,14,17,NaN,NaN,NaN,NaN,0,0,1,0,Кирпичный,Железобетонные,0,0,Центральное,Нет,NaN,NaN,17.0,NaN,17.0,12700000.0
4,2.0,ЗАО,Тропарево-Никулино,65.2,NaN,NaN,6,21,NaN,NaN,Евроремонт,NaN,0,0,0,1,Монолитный,Железобетонные,0,0,Индивидуальный тепловой пункт,Нет,Многоуровневая,NaN,10.0,NaN,NaN,26000000.0


In [5]:
df.shape

(29211, 28)

Сразу разбиваю на тенировочный и валидационный датасеты

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, valid = train_test_split(df, test_size=0.3, random_state=27)

Обрабатываю данные датасета X_train

In [8]:
train.shape

(20447, 28)

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20447 entries, 10086 to 5139
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   n_rooms                  20446 non-null  float64
 1   territorial_division     20447 non-null  object 
 2   district                 20097 non-null  object 
 3   total_area               20447 non-null  float64
 4   living_space             16781 non-null  float64
 5   kitchen_area             17590 non-null  float64
 6   floor                    20447 non-null  int64  
 7   total_floors             20447 non-null  int64  
 8   year_of_built            15064 non-null  float64
 9   celling_height           11872 non-null  float64
 10  renovation_type          18874 non-null  object 
 11  outside_view             15840 non-null  object 
 12  n_balconies              20447 non-null  int64  
 13  n_loggias                20447 non-null  int64  
 14  n_sep_bathrooms    

Удаляю признак mins_to_subway_by_car, т.к. в нем нет ни одного значения

In [11]:
train = train.drop('mins_to_subway_by_car', axis=1)

Вывожу количества значений по каждому признаку

In [13]:
for column in train.columns:
    print(f'Столбец {column}:\n')
    print(train[column].value_counts())
    print('\n' + '*'*30)

Столбец n_rooms:

2.0    7971
3.0    7282
1.0    5187
4.0       4
5.0       2
Name: n_rooms, dtype: int64

******************************
Столбец territorial_division:

 ЦАО                                   3053
 ЗАО                                   2895
 САО                                   2030
 НАО (Новомосковский)                  1927
 ЮАО                                   1807
 СВАО                                  1796
 ЮЗАО                                  1757
 ЮВАО                                  1706
 ВАО                                   1688
 СЗАО                                  1390
 ТАО (Троицкий)                         227
 ЗелАО                                  165
 Раменский городской округ                3
 р-н Крюково                              2
 Сергиево-Посадский городской округ       1
Name: territorial_division, dtype: int64

******************************
Столбец district:

поселение            1365
Пресненский          1004
Раменки               692
Х

Удаляю записи с 4 и 5 комнатами, т.к. их не должно быть в датасете

In [15]:
train = train.loc[train.n_rooms <= 3.0, :]

В territorial_division каждое значение начинается с пробела. Удалю его

In [17]:
def remove_spaces(value):
    return value[1:]

In [18]:
train.territorial_division = train.territorial_division.apply(remove_spaces)

Удаляю записи со значением признака territorial_division "Раменский городской округ" и "Сергиево-Посадский городской округ", т.к. это не Москва

In [19]:
train = train.loc[train.territorial_division != 'Раменский городской округ', :]

In [20]:
train = train.loc[train.territorial_division != 'Сергиево-Посадский городской округ', :]

Удаляю столбцы n_balconies, n_loggias, n_pass_lifts, n_serv_lifts и home_emergency, т.к. они имеют только одно значение и не несут никакой информации

In [21]:
train = train.drop(['n_balconies', 'n_loggias', 'n_pass_lifts', 
                    'n_serv_lifts', 'home_emergency'], axis=1)

### Обработка пропущенных значений

In [22]:
# Количество пропущенных значений по каждому признаку
train.isna().sum()

n_rooms                        0
territorial_division           0
district                     350
total_area                     0
living_space                3662
kitchen_area                2852
floor                          0
total_floors                   0
year_of_built               5379
celling_height              8569
renovation_type             1570
outside_view                4604
n_sep_bathrooms                0
n_comb_bathrooms               0
building_type               2731
overlap_type                6004
heating_type                5693
parking_type               13000
garbage_chute               9215
mins_to_subway_by_walk      4977
mins_to_subway_by_trans    11696
price                          0
dtype: int64

district

In [38]:
print(train.loc[train.territorial_division == 'ТАО (Троицкий)', 'district'].mode()[0])
print(train.loc[train.territorial_division == 'НАО (Новомосковский)', 'district'].mode()[0])

поселение
поселение


In [39]:
# присваиваю пропущенным значениям значениt 'поселение'
train.loc[train.district.isna(), 'district'] = 'поселение'

living_space

In [52]:
train.groupby(by=['total_area'])[['living_space',  'kitchen_area']].median().reset_index()

,total_area,living_space,kitchen_area
0,9.6,NaN,NaN
1,10.1,6.0,3.0
2,10.9,7.0,NaN
3,11.0,NaN,NaN
4,11.2,8.5,2.0
...,...,...,...
1472,299.0,100.0,20.0
1473,360.0,177.0,25.0
1474,440.0,NaN,NaN
1475,443.0,NaN,NaN


In [53]:
median_by_total_area = train.groupby(by=['total_area'])[['living_space', 
                                                         'kitchen_area']].median().reset_index()

In [57]:
median_by_total_area[median_by_total_area.total_area == 97.0]

,total_area,living_space,kitchen_area
922,97.0,54.0,16.0


In [49]:
train.loc[train.living_space.isna()]

,n_rooms,territorial_division,district,total_area,living_space,kitchen_area,floor,total_floors,year_of_built,celling_height,renovation_type,outside_view,n_sep_bathrooms,n_comb_bathrooms,building_type,overlap_type,heating_type,parking_type,garbage_chute,mins_to_subway_by_walk,mins_to_subway_by_trans,price
10086,1.0,НАО (Новомосковский),дп,42.0,NaN,NaN,2,17,2011.0,NaN,Косметический,NaN,1,0,Монолитный,Железобетонные,Центральное,NaN,Нет,NaN,15.0,8000000.0
4789,1.0,ВАО,Гольяново,31.2,NaN,NaN,4,9,1965.0,2.64,Косметический,На улицу и двор,0,1,Блочный,Железобетонные,Центральное,NaN,Есть,9.0,5.0,8700000.0
3453,2.0,ЦАО,Пресненский,97.0,NaN,NaN,10,78,2020.0,NaN,Дизайнерский,На улицу,1,1,Монолитный,NaN,NaN,NaN,NaN,5.0,NaN,58000000.0
28208,2.0,ЮВАО,Рязанский,57.0,NaN,12.0,2,15,NaN,2.50,Без ремонта,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14350000.0
26043,1.0,НАО (Новомосковский),поселок,34.3,NaN,8.0,9,17,2015.0,2.70,Косметический,NaN,1,0,Монолитный,Железобетонные,Центральное,NaN,NaN,NaN,NaN,8499000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11829,3.0,ЗАО,Раменки,170.0,NaN,NaN,3,6,NaN,NaN,Без ремонта,NaN,0,0,Монолитный,NaN,NaN,NaN,NaN,19.0,4.0,118150000.0
19955,3.0,ЗАО,Раменки,144.0,NaN,NaN,3,17,2006.0,3.00,Дизайнерский,Во двор,1,2,Монолитно кирпичный,NaN,NaN,Подземная,Есть,14.0,NaN,62900000.0
14495,2.0,САО,Сокол,65.7,NaN,10.0,4,9,1953.0,NaN,Косметический,Во двор,1,0,Кирпичный,Смешанные,Центральное,NaN,Есть,2.0,NaN,27000000.0
27310,3.0,СВАО,Бутырский,90.4,NaN,NaN,31,46,NaN,3.00,Без ремонта,На улицу и двор,1,0,Монолитный,NaN,NaN,Подземная,NaN,13.0,4.0,29500000.0
